In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("marketing_campaign.csv", sep="\t")

# Data Cleaning & Preprocessing
df_clean = df.copy()
df_clean = df_clean.dropna()
df_clean = df_clean[df_clean['Age'] > 0]
df_clean = df_clean[df_clean['Income'] > 0]

# Tạo các features như trong notebook
df_clean['TotalSpend'] = df_clean['Quantity'] * df_clean['UnitPrice']
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Tính Recency, Frequency, Monetary
snapshot_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalSpend': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Loại bỏ outliers
Q1 = rfm[['Recency', 'Frequency', 'Monetary']].quantile(0.25)
Q3 = rfm[['Recency', 'Frequency', 'Monetary']].quantile(0.75)
IQR = Q3 - Q1

rfm_clean = rfm[~((rfm[['Recency', 'Frequency', 'Monetary']] < (Q1 - 1.5 * IQR)) | 
                  (rfm[['Recency', 'Frequency', 'Monetary']] > (Q3 + 1.5 * IQR))).any(axis=1)]

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_clean[['Recency', 'Frequency', 'Monetary']])
rfm_scaled = pd.DataFrame(rfm_scaled, columns=['Recency', 'Frequency', 'Monetary'])

# Thực hiện clustering (K-means)
kmeans = KMeans(n_clusters=4, random_state=42)
rfm_clean['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Đặt tên cho clusters dựa trên phân tích của bạn
cluster_names = {
    0: 'Champions',
    1: 'At Risk', 
    2: 'Need Attention',
    3: 'Loyal Customers'
}
rfm_clean['Segment'] = rfm_clean['Cluster'].map(cluster_names)

# Tính thêm các metrics cho dashboard
segment_summary = rfm_clean.groupby('Segment').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean', 
    'Monetary': 'mean'
}).reset_index()
segment_summary.columns = ['Segment', 'CustomerCount', 'AvgRecency', 'AvgFrequency', 'AvgMonetary']

# Xuất dữ liệu cho Power BI
rfm_clean.to_csv('powerbi/customer_segments.csv', index=False)
segment_summary.to_csv('powerbi/segment_summary.csv', index=False)

# Xuất thêm dữ liệu RFM scores cho visualization
rfm_scores = rfm_clean.copy()
rfm_scores['RFM_Score'] = rfm_scores['Recency'] + rfm_scores['Frequency'] + rfm_scores['Monetary']

print("Data preparation completed! Files saved for Power BI.")

KeyError: 'Age'

In [ ]:
# File 2: Tạo dữ liệu time series cho trend analysis
daily_sales = df_clean.groupby(pd.to_datetime(df_clean['InvoiceDate']).dt.date).agg({
    'TotalSpend': 'sum',
    'CustomerID': 'nunique',
    'InvoiceNo': 'nunique'
}).reset_index()
daily_sales.columns = ['Date', 'DailyRevenue', 'UniqueCustomers', 'NumberOfInvoices']
daily_sales.to_csv('powerbi/daily_sales.csv', index=False)

# File 3: Dữ liệu sản phẩm
product_analysis = df_clean.groupby('StockCode').agg({
    'Description': 'first',
    'Quantity': 'sum',
    'TotalSpend': 'sum',
    'CustomerID': 'nunique'
}).reset_index()
product_analysis.columns = ['StockCode', 'ProductName', 'TotalQuantity', 'TotalRevenue', 'UniqueCustomers']
product_analysis.to_csv('powerbi/product_analysis.csv', index=False)

# File 4: Dữ liệu geographic (nếu có)
if 'Country' in df_clean.columns:
    country_analysis = df_clean.groupby('Country').agg({
        'CustomerID': 'nunique',
        'TotalSpend': 'sum',
        'InvoiceNo': 'nunique'
    }).reset_index()
    country_analysis.to_csv('powerbi/country_analysis.csv', index=False)